# Basic MCP

ModelContextProtocol (MCP) allows to easily extend application and LLM capabilities using standardized feature implementation. draive library comes with support for MCP both as a server and client allowing to build LLM based application even faster with more code reuse. Lets have a small example:

In [1]:
from collections.abc import AsyncGenerator
from contextlib import asynccontextmanager

from mcp.client.session import ClientSession
from mcp.client.stdio import StdioServerParameters, stdio_client

from draive import (
    ConversationMessage,
    Toolbox,
    conversation_completion,
    ctx,
    load_env,
    setup_logging,
)
from draive.mcp import MCPClient
from draive.openai import OpenAIChatConfig, openai_lmm

load_env() # load .env variables
setup_logging("mcp")

# prepare MCP connection and session
@asynccontextmanager
async def mcp_session() -> AsyncGenerator[ClientSession]:
    async with stdio_client(
        # we are going to use stdio connection within one of the example servers
        StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "/Users/miquido/Desktop",
            ],
        )
    ) as (read, write):
        async with ClientSession(read, write) as session:
            yield session


# initialize dependencies and configuration
async with ctx.scope(
    "mcp",
    openai_lmm(),  # define used LMM to use OpenAI
    OpenAIChatConfig(model="gpt-4o-mini"),  # configure OpenAI model
    # prepare MCPClient, it will handle connection lifetime through context
    # and provide associated state with MCP functionalities
    disposables=[MCPClient(mcp_session())]
):
    # request model using any appropriate method, i.e. conversation for chat
    response: ConversationMessage = await conversation_completion(
        # provide a prompt instruction
        instruction="You can access files on behalf of the user on their machine using available tools."
        " Desktop directory path is `/Users/miquido/Desktop`",
        # add user input
        input="What is on my desktop?",
        # define tools available to the model from MCP extensions
        tools=await Toolbox.external(),
    )
    print(response.content)

14/Jan/2025:16:46:24 +0000 [DEBUG] [mcp] [1d73548024914b4d859339c59e2fc8a5] [mcp] [72f5df3949164e7fa66c1460cdbb9ab2] Entering context...
14/Jan/2025:16:46:24 +0000 [DEBUG] [conversation_completion] [1d73548024914b4d859339c59e2fc8a5] [conversation_completion] [30301b655cbe41bcb69f03b33b463a77] Entering context...
14/Jan/2025:16:46:24 +0000 [DEBUG] [openai_lmm_invocation] [1d73548024914b4d859339c59e2fc8a5] [openai_lmm_invocation] [063d4f2b6cb24fc4a5955aff6bbdb828] Entering context...
14/Jan/2025:16:46:24 +0000 [DEBUG] [httpcore.connection] connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
14/Jan/2025:16:46:24 +0000 [DEBUG] [httpcore.connection] connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x119b52600>
14/Jan/2025:16:46:24 +0000 [DEBUG] [httpcore.connection] start_tls.started ssl_context=<ssl.SSLContext object at 0x119f29150> server_hostname='api.openai.com' timeout=5.0
14/Jan/2025:16:46:24 +0000 [DEB